## Style transfer of Donald Trump's tweets
### A project for the AI course Advanced Natural Language Processing
_Rik Dijkstra, Abel de Wit, Max Knappe_

Every piece of text fits in a specific time, place and scenario, conveys specific characteristics of the user of language and has a specific intent. If we denote the piece of text as `x` and the style of this text as `a`. Text Style Transfer (TST) aims to produce text `x` of a desired attribute value `a`, given the existing text `x'`.

In [1]:
import pandas as pd
import numpy as np
import torch


ModuleNotFoundError: No module named 'pytorch'